# Homework 9:

Repeat the procedure we did in class to map the cities of the US using a map and the results of a PCoA, but using a different country of your choosing.

Tips:

These are the packages that you will need:

```
import geopy.distance
import pandas as pd
import plotly.express as px
import skbio
```

You will also need the `world_cities.csv` file to get the city names and coordinates for your country.

1. Choose your country.
2. Get only the cities of that country from the `world_cities.csv` file.
3. Sort the cities by population and select the 20 to 50 most populated cities.
4. Create a data frame with only these cities.
5. Calculate the distance between the cities using their latitude/longitude data from your data frame (geopy).
6. Create a data frame with a table of all the distances between each paur of cities. This needs to be in a distance matrix format, just like we imported from the `cities.csv` file for the US.
7. Perform the PCoA using skbio.
8. Plot the results of the PCoA using plotly express.
9. Plot the map of the country with the cities using the latitude/longitude from your table using plotly express.
10. That's it!

In [89]:
from geopy.distance import geodesic as gd
import pandas as pd
import plotly.express as px
# from skbio.stats.ordination import pcoa
import pcoa

df = pd.read_csv("world_cities.csv",sep="\t")
df = df.loc[df['country.etc'] == "China"]
df = df.sort_values(by=['pop'], ascending=False)
df = df.head(30)

distmatrix = pd.DataFrame(index=df.name, columns=df.name, dtype=float)
coords = df.loc[:, ["name", "lat", "long"]]
coords['lat'] = pd.to_numeric(coords['lat'])
coords['long'] = pd.to_numeric(coords['long'])

for idx1, row1 in coords.iterrows():  # for each origin
    for idx2, row2 in coords.iterrows():  # for each destination
        distmatrix.loc[row1['name'], row2['name']] = gd((row1['lat'], row1['long']), (row2['lat'], row2['long'])).km

# distmatrix.at['name'] = distmatrix.index
# distmatrix.set_index("name")

my_pcoa = pcoa.pcoa(distmatrix)
my_pcoa['loadings']

fig = px.scatter(my_pcoa['loadings'], x = "PC1", y = "PC2", text=my_pcoa['loadings'].index)
fig = px.scatter_geo(df, lon ='long', lat = 'lat', text="name", size="pop")
fig.update_layout(geo_scope="asia")
fig

